## 05.1 Creating an NFT
##### Peter Gruber, Mattia Biancaterra (mattia.biancaterra@usi.ch, peter.gruber@usi.ch)
2023-01-19

* Simplified NFT

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
import base64
import IPython.display
import hashlib, requests

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()['last-round']

27752228

In [5]:
print(MyAlgo['public'])

33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA


## Showcase

this is an example of NFT and how it looks like on Algorand block explorers

- https://algoexplorer.io/asset/1018480050 
- https://explorer.perawallet.app/assets/1018480050/

## Create a simplified NFT
* In theory, NFTs require extensive metadata and hash information for integrity
* A simplified NFT with only an image link *currently* works with the Pera Algo Wallet
    * A first step for learning
    * **Not recommended**

#### Step 1: the link to the image
* Link is on the Pinata gateway to IPFS
* Instead of filename, use `cid` = *content identifier*
* Test the link (*very slow*)

In [6]:
cid = "QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89"
url = 'https://gateway.pinata.cloud/ipfs/'+cid
print(url)

https://gateway.pinata.cloud/ipfs/QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89


#### Step 2: the NFT transaction
* NFT is based on an ASA
    * `total` (supply) = 1
    * Additional field `url`

##### Step 2.1a: Setup

In [7]:
NFT_name = "Maj NFT"
NFT_unit = "MAJ"
NFT_supply = 1
NFT_decimals = 0
# plus the URL from above

##### Step 2.1b: Same `AssetConfigTxn` as for creating a token
* `MyAlgo` creates mints the NFT

In [8]:
sp=algod_client.suggested_params()

txn = AssetConfigTxn(
    sender=MyAlgo['public'],
    sp=sp,
    total=NFT_supply,
    decimals=NFT_decimals,
    asset_name=NFT_name,
    unit_name=NFT_unit,
    manager=MyAlgo['public'],
    reserve=MyAlgo['public'],
    freeze=MyAlgo['public'],
    clawback=MyAlgo['public'],
    url=url                            # Direct link to file, no metadata
)

#### Step 3: Sign and send

In [9]:
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

CLOQVDOKTJR23NWLPLTO75H43FP6BV6ABBEDNMSHO7LZ7QOQHP4Q


#### Step 4: Wait for confirmation

In [10]:
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  27752296.
Waiting for round 27752296 to finish.
Waiting for round 27752297 to finish.
Transaction CLOQVDOKTJR23NWLPLTO75H43FP6BV6ABBEDNMSHO7LZ7QOQHP4Q confirmed in round 27752298.


#### Step 5: Verification
##### Step 5.1: NFT index
* The NFT `index` is automatically created

In [11]:
NFT_id = txinfo['asset-index']
print(NFT_id)

159493805


##### Step 5.2: Inspect on Pera Explorer and in Pera Wallet
* Inspect on Pera Explorer
* Also inspect in the Pera Wallet (TestNet mode)

In [12]:
print('https://testnet.explorer.perawallet.app/assets/{}'.format(NFT_id))

https://testnet.explorer.perawallet.app/assets/159493805


##### Step 5.3 But there is a catch ...
* Algoexplorer does not recognize our NFT ...
* because it does not satisfy *all* rules for NFTs

In [13]:
print('https://testnet.algoexplorer.io/asset/{}'.format(NFT_id))

https://testnet.algoexplorer.io/asset/159493805


##### Step 6: Check on Blockchain
* NFT is an asset that we hold (like USDC)
* But also an Asset that we have created

In [14]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,3.382550e+01,ALGO,0,Algorand,6
1,1.100000e+02,USDC,10458941,USDC,6
2,0.000000e+00,DRZY,12887013,Drizzy,1
3,9.000000e+02,HUSKEN,159159432,Husky Token,2
4,1.000000e+02,DWSC,159159534,Daniels WSC coin,2
5,9.969900e+09,Beer,159171974,Beer Coin,2
6,1.670000e+01,TEMP,159173248,Peters Tempcoin,1
7,1.000000e+02,TEMPYRY,159173545,Pyrys Tempcoin,1
8,9.170000e+01,TEMPYRY,159173586,Pyrys Tempcoin,1
9,9.900000e+02,WSC,159189398,Peters WSC coin,2


In [15]:
# Looking for assets that we hold
[asset for asset in algod_client.account_info(MyAlgo['public'])['assets'] if asset['asset-id']==NFT_id]

[{'amount': 1, 'asset-id': 159493805, 'is-frozen': False}]

In [16]:
# Looking for asset that we have created
# NOTE: slightly different naming!!
[asset for asset in algod_client.account_info(MyAlgo['public'])['created-assets'] if asset['index']==NFT_id]

[{'index': 159493805,
  'params': {'clawback': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'creator': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'decimals': 0,
   'default-frozen': False,
   'freeze': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'manager': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'name': 'Maj NFT',
   'name-b64': 'TWFqIE5GVA==',
   'reserve': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'total': 1,
   'unit-name': 'MAJ',
   'unit-name-b64': 'TUFK',
   'url': 'https://gateway.pinata.cloud/ipfs/QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89',
   'url-b64': 'aHR0cHM6Ly9nYXRld2F5LnBpbmF0YS5jbG91ZC9pcGZzL1FtWEVLYllKSEtWYlo5WkFWTUpLclgxMmtvWjNidk1Telh1Vkx5cmZVTG9WODk='}}]

## Exercise
* Repeat the entire notebook with a different file format, e.g. `png` or `pdf`
